In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import cv2
import yaml
import sys
import os
from IPython import display
import time
from google.colab import drive
import torch
import torch.nn as nn

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'
from segment_anything.utils.amg import *
from pycocotools import mask as mask_utils

PyTorch version: 2.0.1+cu118
Torchvision version: 0.15.2+cu118
CUDA is available: True
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-_fhv626h
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-_fhv626h
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36589 sha256=496635105cf066e88b7db8f27033a0175a6999f932c9fd2e810b59ccd10ff909
  Stored in directory: /tmp/pip-ephem-wheel-cache-06p7rcno/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything


In [ ]:
folder_path = "/content/drive/MyDrive/Project/Project/data/Arik"
# folder_path = "/content/drive/MyDrive/Project/data/Arik"

In [ ]:
images = ['DJI_20230221091035_0034_V', 'DJI_20230221095213_0011_V', 'DJI_20230221100215_0124_V',
          'DJI_20230221081744_0017_V']
split = 0

In [ ]:
labels_dict = {'Metal (general)':0, 'Metal (iron bender)':0, 'Metal (pipe)':0,
               'Wood (pallett)':1, 'Wood (scraps/cuttings)':1,
               'Carton':2,
               'Concrete':3,
               'Plastic (big bag)':4, 'Plastic (bucket)':4, 'Plastic (general)':4,
               'Plastic (pipe)':4, 'Plastic (sand bag)':4,
               'Nylon':5,
               'Rubber':6,
               'Textil/Fabric/Cloth':7,
               'Background':8}

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [ ]:
img = cv2.imread(f'{folder_path}/{images[0]}/{images[0]}.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
# img = curr_dataset[0][0]
# print(img)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Find the GLCM
import skimage.feature as feature

# Param:
# source image
# List of pixel pair distance offsets - here 1 in each direction
# List of pixel pair angles in radians
graycom = feature.greycomatrix(gray, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256)

# Find the GLCM properties
contrast = feature.greycoprops(graycom, 'contrast')
dissimilarity = feature.greycoprops(graycom, 'dissimilarity')
homogeneity = feature.greycoprops(graycom, 'homogeneity')
energy = feature.greycoprops(graycom, 'energy')
correlation = feature.greycoprops(graycom, 'correlation')
ASM = feature.greycoprops(graycom, 'ASM')

print("Contrast: {}".format(contrast))
print("Dissimilarity: {}".format(dissimilarity))
print("Homogeneity: {}".format(homogeneity))
print("Energy: {}".format(energy))
print("Correlation: {}".format(correlation))
print("ASM: {}".format(ASM))

/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')


Contrast: [[275.08295767 478.6236908  302.40155111 480.78223043]]
Dissimilarity: [[10.53462816 14.21255683 11.08607923 14.14114552]]
Homogeneity: [[0.14030093 0.11010559 0.13737736 0.11163107]]
Energy: [[0.01115822 0.00962345 0.01094506 0.0096626 ]]
Correlation: [[0.95726568 0.92564564 0.95302494 0.92531037]]
ASM: [[1.24505984e-04 9.26107083e-05 1.19794309e-04 9.33658192e-05]]


/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/

In [ ]:
from PIL import Image

class SplitDatasetMaskedResized(Dataset):
  def __init__(self, folder_path, image_name, sp):
    self.image_name = image_name
    self.sp = sp
    self.im_path = f'{folder_path}/{image_name}'
    self.split_path = f'{self.im_path}/split_{sp}'
    with open(f'{self.split_path}/seg_{sp}.yaml') as f:
      self.seg = yaml.safe_load(f)
    files_in_folder = os.listdir(self.split_path)
    self.ann_dict={image_name:{sp:{}}}
    if f'anns_{sp}.yaml' in files_in_folder:
      with open(f'{self.split_path}/anns_{sp}.yaml', 'r') as f:
        self.ann_dict = yaml.safe_load(f)
        # Remove all unknowns
        self.ann_dict[self.image_name][self.sp] = {key:val for key, val in self.ann_dict[self.image_name][self.sp].items() if val != 'Unknown'}
        # TODO: eliminate masks that are too small

  def __len__(self):
    return len(self.ann_dict[self.image_name][self.sp])

  def __getitem__(self, idx):
    annotated_masks = list(self.ann_dict[self.image_name][self.sp].keys())
    m = annotated_masks[idx]
    img = cv2.imread(f'{self.split_path}/{m}.jpg')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    im_pil = Image.fromarray(img)
    preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(im_pil)

    # mask = rle_to_mask(self.seg[self.image_name][self.sp][m]['segmentation'])
    label = self.ann_dict[self.image_name][self.sp][m]
    y = labels_dict[label]
    return input_tensor, y



In [ ]:
arr = []
for image in images:
  for split in range(16):
    ds = SplitDatasetMaskedResized(folder_path, image, split)
    arr.append(ds)

curr_dataset = torch.utils.data.ConcatDataset(arr)



In [ ]:
l = len(curr_dataset)
generator1 = torch.Generator().manual_seed(42)
train_ds, val_ds, test_ds = torch.utils.data.random_split(curr_dataset, [0.6, 0.2, 0.2], generator=generator1)
train_dataloader = DataLoader(train_ds, batch_size=64, shuffle=True)
valid_dataloader = DataLoader(val_ds, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size=64, shuffle=True)


In [ ]:
lab = torch.zeros(9).cuda()
for y in [x[1] for x in curr_dataset]:
    lab[y] += 1
lab = lab/lab.norm()
lab = 1/lab
lab[6] = 0
print(lab)

tensor([72.1773,  4.5802,  5.5521,  2.2507,  8.7947, 22.2675,  0.0000, 55.0827,
         1.1893], device='cuda:0')


In [ ]:
dataset_sizes = {}
dataset_sizes['train'] = len(train_ds)
dataset_sizes['val'] = len(val_ds)
dataset_sizes['test'] = len(test_ds)

dataloaders = {}
dataloaders['train'] = train_dataloader
dataloaders['val'] = valid_dataloader
dataloaders['test'] = test_dataloader


In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

# Best available weights (currently alias for IMAGENET1K_V2)
# Note that these weights may change across versions
original_model = resnet50(weights=ResNet50_Weights.DEFAULT)

In [ ]:
weights = ResNet50_Weights.DEFAULT
# category_name = weights.meta['categories'][label]

In [ ]:
# from transformers import AutoFeatureExtractor, AutoModelForImageClassification

# extractor = AutoFeatureExtractor.from_pretrained("yangy50/garbage-classification")

# model = AutoModelForImageClassification.from_pretrained("yangy50/garbage-classification")

In [ ]:
reverse_labels_dict = {0:'Metal',
                1:'Wood',
               2:'Carton',
               3:'Concrete',
               4:'Plastic',
               5:'Nylon',
               6:'Rubber',
               7:'Textil/Fabric/Cloth',
               8:'Background'}

In [ ]:
running_vloss = 0.0
running_vaccuracy = 0.0
original_model.eval()
# confusion_matrix = torch.zeros(9, 9)
# Disable gradient computation and reduce memory consumption.
with torch.no_grad():
    for i, vdata in enumerate(test_dataloader):
        vinputs, vlabels = vdata
        vlabels = vlabels.cuda()
        # vinputs = vinputs.permute(0, 3, 1, 2)
        # print(vinputs.shape)
        voutputs = original_model(vinputs).cuda()
        # voutputs = model(vinputs)
        # vloss = loss_fn(voutputs, vlabels)
        pred = voutputs.argmax(dim=1)
        for i in range(pred.shape[0]):
          category_name = weights.meta['categories'][pred[i]]
          real = reverse_labels_dict[vlabels[i].item()]
          print(category_name + " vs " + real)
        # running_vaccuracy += (vlabels == pred).float().mean()
        # for t, p in zip(vlabels.view(-1), pred.view(-1)):
        #     confusion_matrix[t.long(), p.long()] += 1
        # per_class_running_accuracy += (vlabels == pred).float()
        # running_vloss += vloss

# avg_vloss = running_vloss / (i + 1)
avg_vaccuracy = running_vaccuracy / (i + 1)

# print('LOSS valid {}'.format(avg_vloss))
print('ACCURACY train valid {}'.format(avg_vaccuracy))
# print('Per class accuracy:',confusion_matrix.diag()/confusion_matrix.sum(1))

In [ ]:
import copy

def train_model(model, criterion, optimizer, scheduler, num_epochs=25, test=False):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            if phase == 'val':
                phase = 'test'
            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.cuda()
                labels = labels.cuda()

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

            if phase == 'test':
                break
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.figure(figsize=(3,3))
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))
# print(classes)
# Make a grid from batch
out = torchvision.utils.make_grid(inputs)
# print(out)
imshow(out, title=[reverse_labels_dict[x.item()] for x in classes])

Transfer Learning: Freezing all layers except final fc

In [ ]:
import torch.optim as optim
from torch.optim import lr_scheduler
# model_conv = torchvision.models.resnet18(weights='DEFAULT')
for param in original_model.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = original_model.fc.in_features
original_model.fc = nn.Linear(num_ftrs, 9)

model = original_model.cuda()

criterion = nn.CrossEntropyLoss(weight=lab) #reweight the loss such that each class has equal contribution to the loss

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 15 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=15, gamma=0.1)

In [ ]:
model_transfered = train_model(model, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=60)

Calc Test accuracy

In [ ]:
train_model(model, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=1, test=True)

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.cuda()
            labels = labels.cuda()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                # plt.figure(figsize=(5,5))
                ax.axis('off')
                ax.set_title('predicted: {}'.format(reverse_labels_dict[preds[j].item()]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
visualize_model(model_transfered, num_images=6)

plt.ioff()
plt.show()

In [ ]:
dataloaders['val']

In [ ]:
for i,(inputs, labels) in enumerate(dataloaders['val']):
  print(i)
  print(inputs, labels)

In [ ]:
dataloaders['val']

In [ ]:
for i in dataloaders['val']:
  print(i)

In [ ]:
with torch.no_grad():
    for i,(inputs, labels) in enumerate(dataloaders['val']):
      inputs = inputs.cuda()
      labels = labels.cuda()
      outputs = model(inputs)
      _, preds = torch.max(outputs, 1)
      break




In [ ]:
class_names = [
              'Metal',
              'Wood',
              'Carton',
              'Concrete',
              'Plastic',
              'Nylon',
              'Rubber',
              'Textil/Fabric/Cloth',
              'Background']


def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img,cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'green'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}%\n({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(len(predictions_array)), class_names, rotation=90)
  plt.yticks([])
  thisplot = plt.bar(range(len(predictions_array)), predictions_array, color="grey")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('green')

In [ ]:
np.array(outputs.cpu().data)[4]

In [ ]:
np.array(labels.cpu().data)

In [ ]:
np.array(inputs.cpu().data)[4]

In [ ]:
i = 0
plt.figure(figsize=(4,2))
plt.subplot(1,2,1)
plot_image(i, np.array(outputs.cpu().data)[i], np.array(labels.cpu().data), np.array(inputs.cpu().data)[i])
plt.subplot(1,2,2)
plot_value_array(i, np.array(outputs.cpu().data)[i],  np.array(labels.cpu().data))
plt.show()

In [ ]:
i = 1
plt.figure(figsize=(4,2))
plt.subplot(1,2,1)
plot_image(i, np.array(outputs.cpu().data)[i], np.array(labels.cpu().data), np.array(inputs.cpu().data)[i])
plt.subplot(1,2,2)
plot_value_array(i, np.array(outputs.cpu().data)[i],  np.array(labels.cpu().data))
plt.show()

In [ ]:
i = 2
plt.figure(figsize=(4,2))
plt.subplot(1,2,1)
plot_image(i, np.array(outputs.cpu().data)[i], np.array(labels.cpu().data), np.array(inputs.cpu().data)[i])
plt.subplot(1,2,2)
plot_value_array(i, np.array(outputs.cpu().data)[i],  np.array(labels.cpu().data))
plt.show()

In [ ]:
num_rows = 1
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
plt.subplots_adjust(wspace=0.1, hspace=2.5)
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, np.array(outputs.cpu().data)[i], np.array(labels.cpu().data), np.array(inputs.cpu().data)[i])
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, np.array(outputs.cpu().data)[i],  np.array(labels.cpu().data))
# plt.tight_layout()
plt.show()

In [ ]:
torch.save(model_transfered.state_dict(), "transfered_basic_resnet50")

Vision Transformer

In [ ]:
# from transformers import AutoFeatureExtractor, AutoModelForImageClassification

# extractor = AutoFeatureExtractor.from_pretrained("yangy50/garbage-classification")

# model = AutoModelForImageClassification.from_pretrained("yangy50/garbage-classification")

In [ ]:
! pip install transformers pytorch-lightning --quiet

In [ ]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
# import pytorch_lightning as pl
from torchmetrics import Accuracy

In [ ]:
from PIL import Image

class SplitDatasetMaskedResizedVIT(Dataset):
  def __init__(self, folder_path, image_name, sp):
    self.image_name = image_name
    self.sp = sp
    self.im_path = f'{folder_path}/{image_name}'
    self.split_path = f'{self.im_path}/split_{sp}'
    with open(f'{self.split_path}/seg_{sp}.yaml') as f:
      self.seg = yaml.safe_load(f)
    files_in_folder = os.listdir(self.split_path)
    self.ann_dict={image_name:{sp:{}}}
    if f'anns_{sp}.yaml' in files_in_folder:
      with open(f'{self.split_path}/anns_{sp}.yaml', 'r') as f:
        self.ann_dict = yaml.safe_load(f)
        # Remove all unknowns
        self.ann_dict[self.image_name][self.sp] = {key:val for key, val in self.ann_dict[self.image_name][self.sp].items() if val != 'Unknown'}

  def __len__(self):
    return len(self.ann_dict[self.image_name][self.sp])

  def __getitem__(self, idx):
    annotated_masks = list(self.ann_dict[self.image_name][self.sp].keys())
    m = annotated_masks[idx]
    img = cv2.imread(f'{self.split_path}/{m}.jpg')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    im_pil = Image.fromarray(img)
    preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(im_pil)
    # mask = rle_to_mask(self.seg[self.image_name][self.sp][m]['segmentation'])
    label = self.ann_dict[self.image_name][self.sp][m]
    y = labels_dict[label]
    return input_tensor, y



In [ ]:
from torchvision.datasets import ImageFolder

In [ ]:
# ds=ImageFolder(folder_path)

In [ ]:
arr = []
for image in images:
  for split in range(16):
    ds = SplitDatasetMaskedResizedVIT(folder_path, image, split)
    # print(ds[0])
    arr.append(ds)

vit_curr_dataset = torch.utils.data.ConcatDataset(arr)

vit_train_ds, vit_val_ds, vit_test_ds = torch.utils.data.random_split(vit_curr_dataset, [0.6, 0.2, 0.2])

In [ ]:
lab = torch.zeros(9).cuda()
for y in [x[1] for x in vit_curr_dataset]:
    lab[y] += 1
lab = lab/lab.norm()
lab = 1/lab
lab[6] = 0
print(lab)

In [ ]:
class ImageClassificationCollator:
    def __init__(self, feature_extractor):
        self.feature_extractor = feature_extractor

    def __call__(self, batch):
        encodings = self.feature_extractor([x[0] for x in batch], return_tensors='pt')
        encodings['labels'] = torch.tensor([x[1] for x in batch], dtype=torch.long)
        return encodings

In [ ]:
feature_extractor = ViTFeatureExtractor(do_normalize=False).from_pretrained('google/vit-base-patch16-224-in21k')

In [ ]:
vit_model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224-in21k',
    num_labels=9,
    label2id={i:x for x, i in reverse_labels_dict.items()},
    id2label=reverse_labels_dict
)
collator = ImageClassificationCollator(feature_extractor)
vit_train_loader = DataLoader(vit_train_ds, batch_size=64, collate_fn=collator, num_workers=2, shuffle=True)
vit_val_loader = DataLoader(vit_val_ds, batch_size=64, collate_fn=collator, num_workers=2)

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(vit_model.parameters(), lr=1e-2)

In [ ]:
from transformers import get_scheduler

num_epochs = 50
num_training_steps = num_epochs * len(vit_train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
vit_model.to(device)

In [ ]:
! pip install datasets

In [ ]:
# from tqdm.auto import tqdm

# progress_bar = tqdm(range(num_training_steps))

# vit_model.train()
# for epoch in range(num_epochs):
#     total_loss = 0
#     total_acc = 0
#     count = 0
#     for batch in vit_train_loader:
#         count += 1
#         batch = {k: v.to(device) for k, v in batch.items()}
#         outputs = vit_model(**batch)
#         loss = outputs.loss
#         loss.backward()
#         total_loss += loss.item()

#         logits = outputs.logits
#         predictions = torch.argmax(logits, dim=-1)
#         total_acc += (predictions == batch['labels']).sum()/len(batch)
#         print(predictions, batch['labels'])
#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         progress_bar.update(1)
#     print("Loss: " + str(total_loss/count) + " Accuracy: " + str(total_acc.item()/count))

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install --upgrade accelerate
!pip install transformers==4.28.0

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-model_basic",
  per_device_train_batch_size=64,
  evaluation_strategy="steps",
  num_train_epochs=50,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-2,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [ ]:
from datasets import load_metric
metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=vit_model,
    args=training_args,
    data_collator=collator,
    compute_metrics=compute_metrics,
    train_dataset=vit_train_ds,
    eval_dataset=vit_val_ds,
    tokenizer=feature_extractor,
)

In [ ]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

In [ ]:
metrics = trainer.evaluate(vit_val_ds)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

In [ ]:
from datasets import load_metric

In [ ]:
metric = load_metric("accuracy")
vit_model.eval()
for batch in vit_val_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = vit_model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()